In [25]:
import pandas as pd
import energy_comms
import numpy as np

In [3]:
employment = pd.read_csv('/Users/mcastillo/Documents/GitHub/rmi-energy-communities/notebooks/files/qualifying_employment_areas.csv')

step 1: get unique areas by geoid

In [4]:
unique_areas = employment.drop_duplicates(subset='geoid')

step 2: keep columns we care about

In [5]:
unique_areas = unique_areas[['area_title','geographic_level','msa_name','geoid']]

In [21]:
unique_areas[unique_areas['geoid']==31420]

,area_title,geographic_level,msa_name,geoid
8,"Macon, GA MSA",metropolitan_stat_area,"Macon, GA",31420


step 3: bring in MSA --> county crosswalk

In [7]:
from energy_comms.extract.msa_codes import extract as msa_codes_extract

In [10]:
msa_codes = msa_codes_extract()

In [22]:
msa_to_county = unique_areas.merge(msa_codes,on='geoid',how='left')

In [24]:
msa_to_county

,area_title,geographic_level,msa_name,geoid,FIPS code,State,State abbreviation,May 2021 MSA code,May 2021 MSA name,County code,Township code,County name (or Township name for the New England states)
0,"Grayson County, Kentucky",county,West Kentucky nonmetropolitan area,21085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Grand Traverse County, Michigan",county,Northwest Lower Peninsula of Michigan nonmetro...,26055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Macon, GA MSA",metropolitan_stat_area,"Macon, GA",31420,13,Georgia,GA,31420.0,"Macon, GA",021,000,Bibb County
3,"Macon, GA MSA",metropolitan_stat_area,"Macon, GA",31420,13,Georgia,GA,31420.0,"Macon, GA",079,000,Crawford County
4,"Macon, GA MSA",metropolitan_stat_area,"Macon, GA",31420,13,Georgia,GA,31420.0,"Macon, GA",169,000,Jones County
...,...,...,...,...,...,...,...,...,...,...,...,...
1055,"New Bern, NC MSA",metropolitan_stat_area,"New Bern, NC",35100,37,North Carolina,NC,35100.0,"New Bern, NC",137,000,Pamlico County
1056,"Homosassa Springs, FL MSA",metropolitan_stat_area,"Homosassa Springs, FL",26140,12,Florida,FL,26140.0,"Homosassa Springs, FL",017,000,Citrus County
1057,"Santa Maria-Santa Barbara, CA MSA",metropolitan_stat_area,"Santa Maria-Santa Barbara, CA",42200,06,California,CA,42200.0,"Santa Maria-Santa Barbara, CA",083,000,Santa Barbara County
1058,"The Villages, FL MSA",metropolitan_stat_area,"The Villages, FL",45540,12,Florida,FL,45540.0,"The Villages, FL",119,000,Sumter County


step 4: make universal fips_county column using logic from geographic level

In [27]:
msa_to_county['fips_county'] = np.where(msa_to_county['geographic_level']=='county',msa_to_county['geoid'],msa_to_county['FIPS code'] + msa_to_county['County code'])


In [30]:
msa_to_county[msa_to_county['fips_county'].isna()]

,area_title,geographic_level,msa_name,geoid,FIPS code,State,State abbreviation,May 2021 MSA code,May 2021 MSA name,County code,Township code,County name (or Township name for the New England states),fips_county


In [31]:
msa_to_county

,area_title,geographic_level,msa_name,geoid,FIPS code,State,State abbreviation,May 2021 MSA code,May 2021 MSA name,County code,Township code,County name (or Township name for the New England states),fips_county
0,"Grayson County, Kentucky",county,West Kentucky nonmetropolitan area,21085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21085
1,"Grand Traverse County, Michigan",county,Northwest Lower Peninsula of Michigan nonmetro...,26055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26055
2,"Macon, GA MSA",metropolitan_stat_area,"Macon, GA",31420,13,Georgia,GA,31420.0,"Macon, GA",021,000,Bibb County,13021
3,"Macon, GA MSA",metropolitan_stat_area,"Macon, GA",31420,13,Georgia,GA,31420.0,"Macon, GA",079,000,Crawford County,13079
4,"Macon, GA MSA",metropolitan_stat_area,"Macon, GA",31420,13,Georgia,GA,31420.0,"Macon, GA",169,000,Jones County,13169
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055,"New Bern, NC MSA",metropolitan_stat_area,"New Bern, NC",35100,37,North Carolina,NC,35100.0,"New Bern, NC",137,000,Pamlico County,37137
1056,"Homosassa Springs, FL MSA",metropolitan_stat_area,"Homosassa Springs, FL",26140,12,Florida,FL,26140.0,"Homosassa Springs, FL",017,000,Citrus County,12017
1057,"Santa Maria-Santa Barbara, CA MSA",metropolitan_stat_area,"Santa Maria-Santa Barbara, CA",42200,06,California,CA,42200.0,"Santa Maria-Santa Barbara, CA",083,000,Santa Barbara County,06083
1058,"The Villages, FL MSA",metropolitan_stat_area,"The Villages, FL",45540,12,Florida,FL,45540.0,"The Villages, FL",119,000,Sumter County,12119


step 5: clean to conform to patio criteria

In [32]:
msa_to_county = msa_to_county[['area_title','fips_county']]

In [34]:
msa_to_county['qualifying_area'] = 'msa_or_county'

msa_to_county['criteria'] = 'fossil_employment'

/var/folders/lw/gjwq5pd52hb01x363x3vyw6c0000gp/T/ipykernel_12351/509814355.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msa_to_county['qualifying_area'] = 'msa_or_county'
/var/folders/lw/gjwq5pd52hb01x363x3vyw6c0000gp/T/ipykernel_12351/509814355.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msa_to_county['criteria'] = 'fossil_employment'


In [35]:
msa_to_county

,area_title,fips_county,qualifying_area,criteria
0,"Grayson County, Kentucky",21085,msa_or_county,fossil_employment
1,"Grand Traverse County, Michigan",26055,msa_or_county,fossil_employment
2,"Macon, GA MSA",13021,msa_or_county,fossil_employment
3,"Macon, GA MSA",13079,msa_or_county,fossil_employment
4,"Macon, GA MSA",13169,msa_or_county,fossil_employment
...,...,...,...,...
1055,"New Bern, NC MSA",37137,msa_or_county,fossil_employment
1056,"Homosassa Springs, FL MSA",12017,msa_or_county,fossil_employment
1057,"Santa Maria-Santa Barbara, CA MSA",06083,msa_or_county,fossil_employment
1058,"The Villages, FL MSA",12119,msa_or_county,fossil_employment
